# Space

In [1]:
import os
import sys 
import logging
import random
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
# pprint(SPACE)

# Available Packages
import argparse
import datasets
import pandas as pd
from datetime import datetime 


from recfldtkn.configfn import load_cohort_args
from recfldtkn.loadtools import load_module_variables, update_args_to_list
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer
from config_observer.CKPD import ckpd_to_CkpdObsConfig

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


g:\Shared drives\CDHAI-WellDoc\2024-WellDocTest-SPACE\_WellDoc-AI-CGMGPT-WorkSpace


# [Part 1] Get a Caseset: Case Examples


## [Step 1] Trigger Event

In [2]:
from recfldtkn.loadtools import fetch_TriggerEvent_tools

####################
# TriggerCaseMethod = 'CGM5MinEntry'
TriggerCaseMethod = 'FoodEntryEOD'
####################

Trigger_tools = fetch_TriggerEvent_tools(TriggerCaseMethod, SPACE)
Trigger_tools 

{'TriggerRecName': 'FoodRec',
 'case_id_columns': ['PID', 'ObsDT'],
 'special_columns': ['PID', 'DT_s'],
 'convert_TriggerEvent_to_Caseset': <function FoodEntryEOD.convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)>}

In [3]:
##################################
CaseSetName = TriggerCaseMethod
case_id_columns = Trigger_tools['case_id_columns']
special_columns = Trigger_tools['special_columns']
##################################

TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)

cohort_args['ckpd_to_CkpdObsConfig'] = ckpd_to_CkpdObsConfig
cohort_args['case_id_columns'] = case_id_columns
cohort_args['ObsDTName'] = 'ObsDT'
cohort_args['PID_ObsDT_columns'] = [cohort_args['RootID'], cohort_args['ObsDTName']]

print(cohort_args)
print(TriggerCasePath)



{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

In [4]:
CaseSetName = 'CGM5MinEntry'
TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
df_case1 = pd.read_pickle(TriggerCasePath) # (1000)

CaseSetName = 'FoodEntryEOD'
TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
df_case2 = pd.read_pickle(TriggerCasePath) # (1000)

df_case = pd.merge(df_case1, df_case2, on=case_id_columns, how='inner')
df_case

,PID,ObsDT
0,1000037,2021-01-07
1,1000037,2021-01-08
2,1000048,2021-01-11
3,1000048,2021-01-13
4,1000048,2021-01-14
...,...,...
2701,3000039,2023-08-06
2702,3000039,2023-08-07
2703,3000039,2023-08-08
2704,3000039,2023-08-11


In [5]:
df_case = df_case.sample(1000, random_state=0).reset_index(drop=True)
ds_case = datasets.Dataset.from_pandas(df_case)
ds_case

Dataset({
    features: ['PID', 'ObsDT'],
    num_rows: 1000
})

## [Step 2] Generate Arguments

In [6]:
#################################
# case_observations = [
#     't1.f1.CGM:ro.CGM5Min-Bf24H-N2Cin1_ct.nsTknIn5MinTS',
#     't2.f1.CGM:ro.CGM5Min-Af2H-N2Cin1_ct.nsTknIn5MinTS',
# ]

case_observations = [
    't1.f1.Time:ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS',

    't2.f1.Time:ro.CGM5Min-Af2H_ct.nsSparseTimeIn5MinTS',
]

name_CaseGamma = 'CatCrossSparseTimeCrossTS'
#################################

In [7]:
from pprint import pprint 
from recfldtkn.obsname import convert_case_observations_to_co_to_observation
from recfldtkn.obsname import get_RecNameList_and_FldTknList

co_to_COName, co_to_CONameInfo = convert_case_observations_to_co_to_observation(case_observations)
co_to_COName

{'t1.f1.Time': 'ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS',
 't2.f1.Time': 'ro.CGM5Min-Af2H_ct.nsSparseTimeIn5MinTS'}

In [8]:
# check whether this information is ready. 
PipelineInfo = get_RecNameList_and_FldTknList(co_to_CONameInfo, ckpd_to_CkpdObsConfig)
pprint(PipelineInfo, sort_dicts=False)

{'RecNameList': ['CGM5Min'],
 'CkpdNameList': ['Af2H', 'Bf24H'],
 'FldTknList': [],
 'CasePhiList': ['nsSparseTimeIn5MinTS']}


In [9]:
COName_List = [CaseName for co, CaseName in co_to_COName.items()]

In [10]:
from recfldtkn.obsname import convert_CONameList_to_CFName

CaseFeatName = convert_CONameList_to_CFName(COName_List, name_CaseGamma)
print(CaseFeatName)

cf.CatCrossSparseTimeCrossTS_co.D20IZBr-Rv


In [11]:
# part of fetch_caseobs_Phi_tools
CF_Folder = os.path.join(SPACE['DATA_CaseFeat'], CaseFeatName)
if not os.path.exists(CF_Folder): os.makedirs(CF_Folder)
print(CF_Folder)

../_Data/3-Data_CaseFeat\cf.CatCrossSparseTimeCrossTS_co.D20IZBr-Rv


# [Part 2] Tools for ds_case_obs

## [Step 1] Prepare examples

In [12]:
case_examples = ds_case[:5]
print(case_examples)

{'PID': [1000672, 1001938, 1001189, 1003551, 1001761], 'ObsDT': [Timestamp('2021-09-04 00:00:00'), Timestamp('2021-08-03 00:00:00'), Timestamp('2021-07-13 00:00:00'), Timestamp('2021-10-01 00:00:00'), Timestamp('2021-07-01 00:00:00')]}


In [13]:
length = len(case_examples[list(case_examples.keys())[0]])
case_examples_list = [{k: v[i] for k, v in case_examples.items()} for i in range(length)]
idx_to_examples = {i: case_examples_list[i] for i in range(length)}
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1000672, 'ObsDT': Timestamp('2021-09-04 00:00:00')},
 1: {'PID': 1001938, 'ObsDT': Timestamp('2021-08-03 00:00:00')},
 2: {'PID': 1001189, 'ObsDT': Timestamp('2021-07-13 00:00:00')},
 3: {'PID': 1003551, 'ObsDT': Timestamp('2021-10-01 00:00:00')},
 4: {'PID': 1001761, 'ObsDT': Timestamp('2021-07-01 00:00:00')}}


In [14]:
case_example = idx_to_examples[0]
case_example

{'PID': 1000672, 'ObsDT': Timestamp('2021-09-04 00:00:00')}

## [Step 2] get_CF_id

In [15]:
# check whether this information is ready. 
PipelineInfo = get_RecNameList_and_FldTknList(co_to_CONameInfo, ckpd_to_CkpdObsConfig)
pprint(PipelineInfo, sort_dicts=False)

{'RecNameList': ['CGM5Min'],
 'CkpdNameList': ['Af2H', 'Bf24H'],
 'FldTknList': [],
 'CasePhiList': ['nsSparseTimeIn5MinTS']}


In [16]:
import inspect
#################################################
def get_CF_id(case_example, case_id_columns, cohort_args):
    # to be update it to a more general function

    ############################## # consider the case_id_columns for DrFirst.
    case_id_columns = cohort_args['case_id_columns'] # ['PID', 'ObsDT', 'PInvID', 'RxID']
    ############################## 

    # case_id_columns = [i for i in case_id_columns if i in case_example]
    li = [col + ':'+ str(case_example[col]) for col in case_id_columns]
    CF_id = '_'.join(li)
    return CF_id 

get_CF_id.fn_string = inspect.getsource(get_CF_id)
#################################################

In [17]:
get_CF_id(case_example, case_id_columns, cohort_args)

'PID:1000672_ObsDT:2021-09-04 00:00:00'

## [Step 3] get_CaseObsName_to_CaseObsInfo

In [18]:
from recfldtkn.observer import get_CaseObsName_to_CaseObsInfo

record_to_ds_rec = {}        # set this to empty dictionary, then we will load data from disk
record_to_ds_rec_info = {}   # set this to empty dictionary, then we will load data from disk
COName_to_COInfo = get_CaseObsName_to_CaseObsInfo(COName_List,
                                                  SPACE, 
                                                  cohort_args, 
                                                  record_to_ds_rec, 
                                                  record_to_ds_rec_info)

In [19]:
for CaseObsName, CaseObsInfo in COName_to_COInfo.items():
    print(CaseObsName)
    print([i for i in CaseObsInfo])

ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS
['RecObsName_List', 'name_CasePhi', 'get_selected_columns', 'CaseObsName', 'ROName_to_ROInfo', 'fn_CasePhi', 'get_CO_id', 'CO_Folder', 'CO_vocab']
ro.CGM5Min-Af2H_ct.nsSparseTimeIn5MinTS
['RecObsName_List', 'name_CasePhi', 'get_selected_columns', 'CaseObsName', 'ROName_to_ROInfo', 'fn_CasePhi', 'get_CO_id', 'CO_Folder', 'CO_vocab']


In [20]:
COName_to_co = {v: k for k, v in co_to_COName.items()}
COName_to_co

{'ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS': 't1.f1.Time',
 'ro.CGM5Min-Af2H_ct.nsSparseTimeIn5MinTS': 't2.f1.Time'}

In [21]:
co_to_COvocab = {COName_to_co[COName]: CaseObsInfo['CO_vocab'] for COName, CaseObsInfo in COName_to_COInfo.items()}

for co, vocab in co_to_COvocab.items():
    print(co)
    print(vocab)

t1.f1.Time
{'tid': {'tid2tkn': {0: 'None', 1: 'Mon1', 2: 'Mon2', 3: 'Mon3', 4: 'Mon4', 5: 'Mon5', 6: 'Mon6', 7: 'Mon7', 8: 'Mon8', 9: 'Mon9', 10: 'Mon10', 11: 'Mon11', 12: 'Mon12', 13: 'WOY1', 14: 'WOY2', 15: 'WOY3', 16: 'WOY4', 17: 'WOY5', 18: 'WOY6', 19: 'WOY7', 20: 'WOY8', 21: 'WOY9', 22: 'WOY10', 23: 'WOY11', 24: 'WOY12', 25: 'WOY13', 26: 'WOY14', 27: 'WOY15', 28: 'WOY16', 29: 'WOY17', 30: 'WOY18', 31: 'WOY19', 32: 'WOY20', 33: 'WOY21', 34: 'WOY22', 35: 'WOY23', 36: 'WOY24', 37: 'WOY25', 38: 'WOY26', 39: 'WOY27', 40: 'WOY28', 41: 'WOY29', 42: 'WOY30', 43: 'WOY31', 44: 'WOY32', 45: 'WOY33', 46: 'WOY34', 47: 'WOY35', 48: 'WOY36', 49: 'WOY37', 50: 'WOY38', 51: 'WOY39', 52: 'WOY40', 53: 'WOY41', 54: 'WOY42', 55: 'WOY43', 56: 'WOY44', 57: 'WOY45', 58: 'WOY46', 59: 'WOY47', 60: 'WOY48', 61: 'WOY49', 62: 'WOY50', 63: 'WOY51', 64: 'WOY52', 65: 'WOY53', 66: 'Weekday0', 67: 'Weekday1', 68: 'Weekday2', 69: 'Weekday3', 70: 'Weekday4', 71: 'Weekday5', 72: 'Weekday6', 73: 'Hour0', 74: 'Hour1', 7

## [Step 3] get_CF_vocab

In [22]:
##################################
def get_CF_vocab(co_to_COvocab):
    CF_vocab = {}
    co_list = [i for i in co_to_COvocab]

    # across all fields
    f_to_coname = {}
    for co in co_list:
        f_list = [i for i in co.split('.') if i[0] == 'f']
        assert len(f_list) == 1
        f = f_list[0]
        f_to_coname[f] = co 
    # print(f_to_coname)
        
    SeqType = 'input_ids'
    idx2tkn_all = ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]',  '[BOS]', '[EOS]']
    for f, co in f_to_coname.items():
        CaseObsVocab = co_to_COvocab[co] 
        # idx2tkn = [f+':'+ tkn for tid, tkn in CaseObsVocab['tid']['tid2tkn'].items()]
        idx2tkn = [tkn for tid, tkn in CaseObsVocab['tid']['tid2tkn'].items()]
        idx2tkn_all = idx2tkn_all + idx2tkn
        break 

    
    tid2tkn_all = {i: tkn for i, tkn in enumerate(idx2tkn_all)}
    tkn2tid_all = {tkn: i for i, tkn in enumerate(idx2tkn_all)}
    CF_vocab[SeqType] = {'tid2tkn': tid2tkn_all, 'tkn2tid': tkn2tid_all}
    
    # across all wgts. pass.
    CF_vocab['input_wgts'] = {}

    # across all timesteps 
    SeqType = 'timestep_ids'
    CF_vocab[SeqType] = {}
    t_to_coname = {}
    for co in co_list:
        t_list = [i for i in co.split('.') if i[0] == 't']
        assert len(t_list) == 1
        t = t_list[0]
        t_to_coname[t] = co 
    
    
    # SeqType = 'timestep'
    # idx2tkn_all = ['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]', '[BOS]', '[EOS]']
    idx2tkn_all = []
    for t, co in t_to_coname.items():
        CaseObsVocab = co_to_COvocab[co] 
        idx2tkn = [t+':'+ tkn for tid, tkn in CaseObsVocab['timestep']['tid2tkn'].items()]
        idx2tkn_all = idx2tkn_all + idx2tkn
    tid2tkn_all = {i: tkn for i, tkn in enumerate(idx2tkn_all)}
    tkn2tid_all = {tkn: i for i, tkn in enumerate(idx2tkn_all)}
    CF_vocab[SeqType] = {'tid2tkn': tid2tkn_all, 'tkn2tid': tkn2tid_all}
    
    # SeqType_list = [i for i in CF_vocab]
    return CF_vocab
##################################

get_CF_vocab.fn_string = inspect.getsource(get_CF_vocab)

In [23]:
CF_vocab = get_CF_vocab(co_to_COvocab)
for SeqType, vocab in CF_vocab.items():
    print(SeqType, ':', vocab)
    print('\n')


pprint(CF_vocab, sort_dicts=False, compact=True)

input_ids : {'tid2tkn': {0: '[PAD]', 1: '[UNK]', 2: '[CLS]', 3: '[SEP]', 4: '[MASK]', 5: '[BOS]', 6: '[EOS]', 7: 'None', 8: 'Mon1', 9: 'Mon2', 10: 'Mon3', 11: 'Mon4', 12: 'Mon5', 13: 'Mon6', 14: 'Mon7', 15: 'Mon8', 16: 'Mon9', 17: 'Mon10', 18: 'Mon11', 19: 'Mon12', 20: 'WOY1', 21: 'WOY2', 22: 'WOY3', 23: 'WOY4', 24: 'WOY5', 25: 'WOY6', 26: 'WOY7', 27: 'WOY8', 28: 'WOY9', 29: 'WOY10', 30: 'WOY11', 31: 'WOY12', 32: 'WOY13', 33: 'WOY14', 34: 'WOY15', 35: 'WOY16', 36: 'WOY17', 37: 'WOY18', 38: 'WOY19', 39: 'WOY20', 40: 'WOY21', 41: 'WOY22', 42: 'WOY23', 43: 'WOY24', 44: 'WOY25', 45: 'WOY26', 46: 'WOY27', 47: 'WOY28', 48: 'WOY29', 49: 'WOY30', 50: 'WOY31', 51: 'WOY32', 52: 'WOY33', 53: 'WOY34', 54: 'WOY35', 55: 'WOY36', 56: 'WOY37', 57: 'WOY38', 58: 'WOY39', 59: 'WOY40', 60: 'WOY41', 61: 'WOY42', 62: 'WOY43', 63: 'WOY44', 64: 'WOY45', 65: 'WOY46', 66: 'WOY47', 67: 'WOY48', 68: 'WOY49', 69: 'WOY50', 70: 'WOY51', 71: 'WOY52', 72: 'WOY53', 73: 'Weekday0', 74: 'Weekday1', 75: 'Weekday2', 76: 'W

## [Step 4] Fetch $o_{ij}$ List

In [24]:
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1000672, 'ObsDT': Timestamp('2021-09-04 00:00:00')},
 1: {'PID': 1001938, 'ObsDT': Timestamp('2021-08-03 00:00:00')},
 2: {'PID': 1001189, 'ObsDT': Timestamp('2021-07-13 00:00:00')},
 3: {'PID': 1003551, 'ObsDT': Timestamp('2021-10-01 00:00:00')},
 4: {'PID': 1001761, 'ObsDT': Timestamp('2021-07-01 00:00:00')}}


In [25]:
use_CO_from_disk = False
COName_to_FnCaseObsPhi = {}
for COName, COInfo in COName_to_COInfo.items():
    ro_to_ROName = None 
    FnCaseObsPhi = CaseObserverTransformer(ro_to_ROName,
                                            COInfo['ROName_to_ROInfo'], 
                                            COInfo['name_CasePhi'], 
                                            COInfo['fn_CasePhi'], 
                                            COInfo['CO_vocab'], 
                                            COInfo['get_CO_id'],
                                            cohort_args,
                                            COInfo['CO_Folder'], 
                                            df_case,
                                            use_CO_from_disk)
    COName_to_FnCaseObsPhi[COName] = FnCaseObsPhi

In [26]:
from recfldtkn.observer import CaseFeatureTransformer

fetch_examples_with_complete_COs = CaseFeatureTransformer.fetch_examples_with_complete_COs

use_CO_from_disk = False 
results = fetch_examples_with_complete_COs(idx_to_examples, 
                                            COName_to_co, 
                                            COName_to_COInfo, 
                                            COName_to_FnCaseObsPhi)

idx_to_examples, COName_to_FnCaseObsPhi = results

In [27]:
case_observations

['t1.f1.Time:ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS',
 't2.f1.Time:ro.CGM5Min-Af2H_ct.nsSparseTimeIn5MinTS']

In [28]:
for i, example in idx_to_examples.items():
    print(i)
    print(example)

0
{'PID': 1000672, 'ObsDT': Timestamp('2021-09-04 00:00:00'), 't1.f1.Time_timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize', 'TimeUnit', 'StartIdx-To-EndIdx'], 't1.f1.Time_timevalues': ['2021-09-04T00:00:00', '5Min', '5', 'minutes', '-288:To:0'], 't2.f1.Time_timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize', 'TimeUnit', 'StartIdx-To-EndIdx'], 't2.f1.Time_timevalues': ['2021-09-04T00:00:00', '5Min', '5', 'minutes', '1:To:24']}
1
{'PID': 1001938, 'ObsDT': Timestamp('2021-08-03 00:00:00'), 't1.f1.Time_timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize', 'TimeUnit', 'StartIdx-To-EndIdx'], 't1.f1.Time_timevalues': ['2021-08-03T00:00:00', '5Min', '5', 'minutes', '-288:To:0'], 't2.f1.Time_timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize', 'TimeUnit', 'StartIdx-To-EndIdx'], 't2.f1.Time_timevalues': ['2021-08-03T00:00:00', '5Min', '5', 'minutes', '1:To:24']}
2
{'PID': 1001189, 'ObsDT': Timestamp('2021-07-13 00:00:00'), 't1.f1.Time_timeinfo': ['ObsDTValue', 'TimeStepTyp

In [29]:
[i for i in example]

['PID',
 'ObsDT',
 't1.f1.Time_timeinfo',
 't1.f1.Time_timevalues',
 't2.f1.Time_timeinfo',
 't2.f1.Time_timevalues']

## [Step 3]* Develop $\Gamma$. (To Dev)

In [30]:
idx = 1
case_example = idx_to_examples[idx]
# print('case_example:', case_example)
pprint(case_example, sort_dicts=False, compact = True)

{'PID': 1001938,
 'ObsDT': Timestamp('2021-08-03 00:00:00'),
 't1.f1.Time_timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize',
                         'TimeUnit', 'StartIdx-To-EndIdx'],
 't1.f1.Time_timevalues': ['2021-08-03T00:00:00', '5Min', '5', 'minutes',
                           '-288:To:0'],
 't2.f1.Time_timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize',
                         'TimeUnit', 'StartIdx-To-EndIdx'],
 't2.f1.Time_timevalues': ['2021-08-03T00:00:00', '5Min', '5', 'minutes',
                           '1:To:24']}


In [31]:
result_case = {}
co_list = [i for i in co_to_COvocab]
co_list

['t1.f1.Time', 't2.f1.Time']

In [32]:
SeqType_list = [i for i in CF_vocab]
SeqType_list

['input_ids', 'input_wgts', 'timestep_ids']

In [33]:
co_list = [i for i in co_to_COvocab]

# across all fields
f_to_coname = {}
for co in co_list:
    f_list = [i for i in co.split('.') if i[0] == 'f']
    assert len(f_list) == 1
    f = f_list[0]
    if f in f_to_coname:
        f_to_coname[f].append(co)
    else:
        f_to_coname[f] = [co]
# print(f_to_coname)
f_to_coname

{'f1': ['t1.f1.Time', 't2.f1.Time']}

In [34]:
t_to_coname = {}
for co in co_list:
    t_list = [i for i in co.split('.') if i[0] == 't']
    assert len(t_list) == 1
    t = t_list[0]
    if t in t_to_coname:
        t_to_coname[t].append(co)
    else:
        t_to_coname[t] = [co]
t_to_coname

{'t1': ['t1.f1.Time'], 't2': ['t2.f1.Time']}

In [35]:
time_period = 't1'

co_list = t_to_coname[time_period]
co_list

['t1.f1.Time']

In [36]:
[i for i in case_example]

['PID',
 'ObsDT',
 't1.f1.Time_timeinfo',
 't1.f1.Time_timevalues',
 't2.f1.Time_timeinfo',
 't2.f1.Time_timevalues']

In [37]:
f_to_coname

{'f1': ['t1.f1.Time', 't2.f1.Time']}

In [38]:
t_to_coname

{'t1': ['t1.f1.Time'], 't2': ['t2.f1.Time']}

In [39]:
[i for i in case_example]

['PID',
 'ObsDT',
 't1.f1.Time_timeinfo',
 't1.f1.Time_timevalues',
 't2.f1.Time_timeinfo',
 't2.f1.Time_timevalues']

In [40]:
final_d = {}
for TimePeriod in t_to_coname:


    co_list = t_to_coname[TimePeriod]
    assert len(co_list) == 1

    co = co_list[0]
    values = {}
    values[f'{co}_timeinfo']   = [timestep for timestep in case_example[f'{co}_timeinfo']]
    values[f'{co}_timevalues'] = [timestep for timestep in case_example[f'{co}_timevalues']]
    df = pd.DataFrame(values) # .rename(columns={f'{co}_{timestepName}': timestepName}) 
    # df   
    d = dict(zip(values[f'{co}_timeinfo'], values[f'{co}_timevalues']))
    d['StartIdx-To-EndIdx' + '_' + TimePeriod] = d.pop('StartIdx-To-EndIdx')
    for k, v in d.items(): final_d[k] = v
    # start = d['StartIdx']
    # end = d['EndIdx']

final_d

{'ObsDTValue': '2021-08-03T00:00:00',
 'TimeStepType': '5Min',
 'TimeStepSize': '5',
 'TimeUnit': 'minutes',
 'StartIdx-To-EndIdx_t1': '-288:To:0',
 'StartIdx-To-EndIdx_t2': '1:To:24'}

In [41]:
CF = {}
CF['timeinfo'] = [i for i in final_d]
CF['timevalues'] = [final_d[i] for i in final_d]
CF

{'timeinfo': ['ObsDTValue',
  'TimeStepType',
  'TimeStepSize',
  'TimeUnit',
  'StartIdx-To-EndIdx_t1',
  'StartIdx-To-EndIdx_t2'],
 'timevalues': ['2021-08-03T00:00:00',
  '5Min',
  '5',
  'minutes',
  '-288:To:0',
  '1:To:24']}

In [42]:
import inspect 

##################################
def fn_CaseGamma(case_example, co_to_COvocab, CF_vocab, cohort_args):
    import itertools
    co_list = [i for i in co_to_COvocab]

    ############### f_to_coname ###############
    f_to_coname = {}
    for co in co_list:
        f_list = [i for i in co.split('.') if i[0] == 'f']
        assert len(f_list) == 1
        f = f_list[0]
        if f in f_to_coname:
            f_to_coname[f].append(co)
        else:
            f_to_coname[f] = [co]
    
    ############### t_to_coname ###############
    t_to_coname = {}
    for co in co_list:
        t_list = [i for i in co.split('.') if i[0] == 't']
        assert len(t_list) == 1
        t = t_list[0]
        if t in t_to_coname:
            t_to_coname[t].append(co)
        else:
            t_to_coname[t] = [co]

    ############### X_tkn_final ###############
    
    final_d = {}
    for TimePeriod in t_to_coname:
        co_list = t_to_coname[TimePeriod]
        assert len(co_list) == 1

        co = co_list[0]
        values = {}
        values[f'{co}_timeinfo']   = [timestep for timestep in case_example[f'{co}_timeinfo']]
        values[f'{co}_timevalues'] = [timestep for timestep in case_example[f'{co}_timevalues']]
        df = pd.DataFrame(values) # .rename(columns={f'{co}_{timestepName}': timestepName}) 
        # df   
        d = dict(zip(values[f'{co}_timeinfo'], values[f'{co}_timevalues']))
        d['StartIdx-To-EndIdx' + '_' + TimePeriod] = d.pop('StartIdx-To-EndIdx')
        for k, v in d.items(): final_d[k] = v

    CF = {}
    CF['timeinfo'] = [i for i in final_d]
    CF['timevalues'] = [final_d[i] for i in final_d]

    return CF  
##################################

fn_CaseGamma.fn_string = inspect.getsource(fn_CaseGamma)

In [43]:
CF = fn_CaseGamma(case_example, co_to_COvocab, CF_vocab, cohort_args)
pprint(CF, compact = True)

{'timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize', 'TimeUnit',
              'StartIdx-To-EndIdx_t1', 'StartIdx-To-EndIdx_t2'],
 'timevalues': ['2021-08-03T00:00:00', '5Min', '5', 'minutes', '-288:To:0',
                '1:To:24']}


## Save to Files

In [44]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = [
    'import pandas as pd', 
    'import numpy as np',
    'import itertools',
    'from functools import reduce',
    ]
fn_variables = [
    get_CF_id, 
    get_CF_vocab, 
    # concatenate_field_from_same_timeperiod, 
    fn_CaseGamma
    ]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_casegamma', f'gamma_{name_CaseGamma}.py')
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# Application

In [45]:
CaseFeatName

'cf.CatCrossSparseTimeCrossTS_co.D20IZBr-Rv'

In [46]:
from recfldtkn.observer import get_CaseFeatInfo_for_a_CaseFeatName


record_to_ds_rec = {}
record_to_ds_rec_info = {}

CaseFeatInfo = get_CaseFeatInfo_for_a_CaseFeatName(name_CaseGamma,
                                                   case_observations,
                                                   SPACE, 
                                                   cohort_args, 
                                                   record_to_ds_rec, 
                                                   record_to_ds_rec_info)
[i for i in CaseFeatInfo]

['CaseFeatName',
 'name_CaseGamma',
 'case_observations',
 'co_to_COName',
 'COName_to_COInfo',
 'COName_List',
 'PipelineInfo',
 'fn_CaseGamma',
 'get_CF_id',
 'get_CF_vocab',
 'CF_vocab',
 'CF_Folder']

In [47]:
import random 
from pprint import pprint
from datetime import datetime 
from recfldtkn.observer import CaseFeatureTransformer

In [48]:

co_to_COName = CaseFeatInfo['co_to_COName']
COName_to_COInfo = CaseFeatInfo['COName_to_COInfo']
name_CaseGamma = CaseFeatInfo['name_CaseGamma']
fn_CaseGamma = CaseFeatInfo['fn_CaseGamma']
CF_vocab = CaseFeatInfo['CF_vocab']
get_CF_id = CaseFeatInfo['get_CF_id']
CF_Folder = CaseFeatInfo['CF_Folder']
df_case = ds_case.to_pandas()
use_CF_from_disk = False
use_CO_from_disk = False
batch_size = CaseFeatInfo.get('batch_size', 1000)

FnCaseFeatGamma = CaseFeatureTransformer(co_to_COName,
                                          COName_to_COInfo, 
                                          name_CaseGamma, 
                                          fn_CaseGamma, 
                                          CF_vocab, 
                                          get_CF_id,
                                          cohort_args,
                                          CF_Folder, 
                                          df_case,
                                          use_CF_from_disk,
                                          use_CO_from_disk)

# Check Cache Functions

In [49]:
FnCaseFeatGamma.new_CFs

{}

In [50]:
start = datetime.now()
batch_size = 100
ds_casetkn = ds_case.map(FnCaseFeatGamma, 
                         batched = True, 
                         batch_size = batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseFeatName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Elipse Time:  0:00:03.091203
Dataset({
    features: ['PID', 'ObsDT', 'timeinfo', 'timevalues'],
    num_rows: 1000
})


In [51]:
start = datetime.now()
batch_size = 100
ds_casetkn = ds_case.map(FnCaseFeatGamma, 
                         batched = True, 
                         batch_size = batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseFeatName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Elipse Time:  0:00:00.023002
Dataset({
    features: ['PID', 'ObsDT', 'timeinfo', 'timevalues'],
    num_rows: 1000
})


In [52]:
print(CaseObsName)
random_int = random.randint(0, len(ds_casetkn))
print(random_int)
pprint(ds_casetkn[random_int])
print(CF_vocab)

ro.CGM5Min-Af2H_ct.nsSparseTimeIn5MinTS
740
{'ObsDT': Timestamp('2021-12-06 00:00:00'),
 'PID': 1001761,
 'timeinfo': ['ObsDTValue',
              'TimeStepType',
              'TimeStepSize',
              'TimeUnit',
              'StartIdx-To-EndIdx_t1',
              'StartIdx-To-EndIdx_t2'],
 'timevalues': ['2021-12-06T00:00:00',
                '5Min',
                '5',
                'minutes',
                '-288:To:0',
                '1:To:24']}
{'input_ids': {'tid2tkn': {0: '[PAD]', 1: '[UNK]', 2: '[CLS]', 3: '[SEP]', 4: '[MASK]', 5: '[BOS]', 6: '[EOS]', 7: 'None', 8: 'Mon1', 9: 'Mon2', 10: 'Mon3', 11: 'Mon4', 12: 'Mon5', 13: 'Mon6', 14: 'Mon7', 15: 'Mon8', 16: 'Mon9', 17: 'Mon10', 18: 'Mon11', 19: 'Mon12', 20: 'WOY1', 21: 'WOY2', 22: 'WOY3', 23: 'WOY4', 24: 'WOY5', 25: 'WOY6', 26: 'WOY7', 27: 'WOY8', 28: 'WOY9', 29: 'WOY10', 30: 'WOY11', 31: 'WOY12', 32: 'WOY13', 33: 'WOY14', 34: 'WOY15', 35: 'WOY16', 36: 'WOY17', 37: 'WOY18', 38: 'WOY19', 39: 'WOY20', 40: 'WOY21', 41

In [53]:
ds_casetkn[72]

{'PID': 1000692,
 'ObsDT': Timestamp('2021-10-10 00:00:00'),
 'timeinfo': ['ObsDTValue',
  'TimeStepType',
  'TimeStepSize',
  'TimeUnit',
  'StartIdx-To-EndIdx_t1',
  'StartIdx-To-EndIdx_t2'],
 'timevalues': ['2021-10-10T00:00:00',
  '5Min',
  '5',
  'minutes',
  '-288:To:0',
  '1:To:24']}

# Save

In [54]:
FnCaseFeatGamma.df_CF_info

,casefeat_idx_in_data
casefeat_id,


In [55]:
CF_Folder_data = FnCaseFeatGamma.CF_Folder_data
CF_Folder_data

'../_Data/3-Data_CaseFeat\\cf.CatCrossSparseTimeCrossTS_co.D20IZBr-Rv\\data'

In [56]:
FnCaseFeatGamma.save_new_CFs_to_disk(CF_Folder_data)

In [57]:
FnCaseFeatGamma.df_CF_info

,casefeat_idx_in_data
casefeat_id,


In [58]:
FnCaseFeatGamma.ds_CF_data

In [59]:
CFids = FnCaseFeatGamma.CFids
ds_CF_data, df_CF_info = FnCaseFeatGamma.load_CFs_from_disk(CF_Folder_data, CFids)

In [60]:
ds_CF_data

In [61]:
df_CF_info

,casefeat_idx_in_data
casefeat_id,


# Save Vocab

In [62]:
CF_vocab = FnCaseFeatGamma.CF_vocab
CF_vocab

{'input_ids': {'tid2tkn': {0: '[PAD]',
   1: '[UNK]',
   2: '[CLS]',
   3: '[SEP]',
   4: '[MASK]',
   5: '[BOS]',
   6: '[EOS]',
   7: 'None',
   8: 'Mon1',
   9: 'Mon2',
   10: 'Mon3',
   11: 'Mon4',
   12: 'Mon5',
   13: 'Mon6',
   14: 'Mon7',
   15: 'Mon8',
   16: 'Mon9',
   17: 'Mon10',
   18: 'Mon11',
   19: 'Mon12',
   20: 'WOY1',
   21: 'WOY2',
   22: 'WOY3',
   23: 'WOY4',
   24: 'WOY5',
   25: 'WOY6',
   26: 'WOY7',
   27: 'WOY8',
   28: 'WOY9',
   29: 'WOY10',
   30: 'WOY11',
   31: 'WOY12',
   32: 'WOY13',
   33: 'WOY14',
   34: 'WOY15',
   35: 'WOY16',
   36: 'WOY17',
   37: 'WOY18',
   38: 'WOY19',
   39: 'WOY20',
   40: 'WOY21',
   41: 'WOY22',
   42: 'WOY23',
   43: 'WOY24',
   44: 'WOY25',
   45: 'WOY26',
   46: 'WOY27',
   47: 'WOY28',
   48: 'WOY29',
   49: 'WOY30',
   50: 'WOY31',
   51: 'WOY32',
   52: 'WOY33',
   53: 'WOY34',
   54: 'WOY35',
   55: 'WOY36',
   56: 'WOY37',
   57: 'WOY38',
   58: 'WOY39',
   59: 'WOY40',
   60: 'WOY41',
   61: 'WOY42',
   62: 'WOY4

In [63]:
CF_Folder_vocab = FnCaseFeatGamma.CF_Folder_vocab
CF_Folder_vocab

'../_Data/3-Data_CaseFeat\\cf.CatCrossSparseTimeCrossTS_co.D20IZBr-Rv\\vocab.p'

In [64]:
df_Vocab = pd.DataFrame({CaseObsName: CF_vocab})
df_Vocab.to_pickle(CF_Folder_vocab)
df_Vocab

,ro.CGM5Min-Af2H_ct.nsSparseTimeIn5MinTS
input_ids,"{'tid2tkn': {0: '[PAD]', 1: '[UNK]', 2: '[CLS]..."
input_wgts,{}
timestep_ids,"{'tid2tkn': {0: 't1:5Min_-288', 1: 't1:5Min_-2..."
